# Step 2 of the NBRA workflow with  DFTB+

In this tutorial, we demonstrate how to compute time-overlaps and NACs in the single-particle basis using DFTB+


## Table of contents
<a name="toc"></a>
1. [Importing needed libraries](#1)
2. [Calculations of energies, MOs and MO-basis time-overlaps](#2)
3. [NBRA workflow](#3)


### A. Learning objectives

* To be able to conduct DFTB+ calculations using Libra auxiliary functions
* To be able to compute time-overlaps in AO and MO basis with DFTB+ orbitals
* To be able to compute NACs, time-overlaps, and vibronic Hamiltonians for multiple time-steps


### B. Use cases

* [process the DFTB+ calculations results](#use-1)
* [compute wavefunction time-overlaps with DFTB+](#use-2)


### C. Functions

- `libra_py`
  - `workflows`
    - `nbra`
      - `step2_dftb`
        - [`do_step`](#do_step-1)
        - [`do_ovlp`](#do_ovlp-1)
        - [`run_step2`](#run_step2-1)
        - [`run_step2_lz`](#run_step2_lz-1)
            

## 1. Importing needed libraries <a name="1"></a>
[Back to TOC](#toc)


In [1]:
import os
import sys

# Fisrt, we add the location of the library to test to the PYTHON path
if sys.platform=="cygwin":
    from cyglibra_core import *
elif sys.platform=="linux" or sys.platform=="linux2":
    from liblibra_core import *
from libra_py.packages.dftbplus import methods as DFTB_methods
import libra_py.workflows.nbra.step2_dftb as step2
from libra_py import units


/home/alexvakimov/SOFTWARE/Conda/envs/libra/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: to-Python converter for std::vector<std::vector<int, std::allocator<int> >, std::allocator<std::vector<int, std::allocator<int> > > > already registered; second conversion method ignored.
  return f(*args, **kwds)
/home/alexvakimov/SOFTWARE/Conda/envs/libra/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: to-Python converter for boost::python::detail::container_element<std::vector<std::vector<int, std::allocator<int> >, std::allocator<std::vector<int, std::allocator<int> > > >, unsigned long, boost::python::detail::final_vector_derived_policies<std::vector<std::vector<int, std::allocator<int> >, std::allocator<std::vector<int, std::allocator<int> > > >, false> > already registered; second conversion method ignored.
  return f(*args, **kwds)
/home/alexvakimov/SOFTWARE/Conda/envs/libra/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: to-Python converter for std::vect

## 2. Calculations of energies, MOs and MO-basis time-overlaps<a name="2"></a>
[Back to TOC](#toc)

**Important** before running any of the below DFTB+ calculations, make sure you have the DFTB+ package and corresponding parameters installed.

First, we will demonstrate doing a single step of MD or NA-MD calculations with DFTB+ using the `do_step` function.

Here, we first do the SCF calculations of the geometry 0 and obtain corresponding orbital energies, molecular orbitals, Hamiltonian and AO overlaps matrix (at a given geometry)
<a name="do_step-1"></a>

In [2]:
params = { "EXE":"/home/alexvakimov/SOFTWARE/dftbplus/dftbplus_22.2/_install/bin/dftb+", 
           "mo_active_space":[26,27,28,29,30], 
           "md_file":"adamantane-md.xyz",
           "sp_gen_file": "x1.gen",
           "ovlp_gen_file": "x2.gen",
           "syst_spec" : "C", 
           "scf_in_file" : "dftb_in_ham1.hsd", 
           "hs_in_file" : "dftb_in_ham2.hsd",
           "ovlp_in_file" : "dftb_in_overlaps.hsd",
           "do_tddftb": False
         }

E0, MO0, H0, S0 = step2.do_step(0, params)

|===============================================================================
|
|  DFTB+ development version (commit: 3f79963e, base: 22.2)
|
|  Copyright (C) 2006 - 2022  DFTB+ developers group
|
|===============================================================================
|
|  When publishing results obtained with DFTB+, please cite the following
|  reference:
|
|  * DFTB+, a software package for efficient approximate density functional
|    theory based atomistic simulations, J. Chem. Phys. 152, 124101 (2020).
|    [doi: 10.1063/1.5143190]
|
|  You should also cite additional publications crediting the parametrization
|  data you use. Please consult the documentation of the SK-files for the
|  references.
|
|===============================================================================

Reading input file 'dftb_in.hsd'
Parser version: 13

--------------------------------------------------------------------------------
***  Converting input from parser version  4 to parser ver

ERROR STOP 

Error termination. Backtrace:
#0  0x560c65d3ba8e in __dftbp_common_globalenv_MOD_abortprogram
	at /home/alexvakimov/SOFTWARE/dftbplus/dftbplus_22.2/src/dftbp/common/globalenv.F90:195
#1  0x560c65e0ba9a in __dftbp_dftbplus_mainio_MOD_writehsandstop
	at /home/alexvakimov/SOFTWARE/dftbplus/dftbplus_22.2/src/dftbp/dftbplus/mainio.F90:4166
#2  0x560c65df2547 in __dftbp_dftbplus_main_MOD_processgeometry
	at /home/alexvakimov/SOFTWARE/dftbplus/dftbplus_22.2/src/dftbp/dftbplus/main.F90:1262
#3  0x560c65df7351 in __dftbp_dftbplus_main_MOD_rundftbplus
	at /home/alexvakimov/SOFTWARE/dftbplus/dftbplus_22.2/src/dftbp/dftbplus/main.F90:236
#4  0x560c65d3ab85 in dftbplus
	at /home/alexvakimov/SOFTWARE/dftbplus/dftbplus_22.2/app/dftb+/dftbplus.F90:35
#5  0x560c65d35fae in main
	at /home/alexvakimov/SOFTWARE/dftbplus/dftbplus_22.2/app/dftb+/dftbplus.F90:11


Some extra documentation on the function is found here:

In [3]:
help(step2.do_step)

Help on function do_step in module libra_py.workflows.nbra.step2_dftb:

do_step(snap, params)
    Runs a single-point SCF calculation for a given geometry along a trajectory
    
    Args:
        snap ( int ): index of the time step to be used from the trajectory file
        params ( dictionary ): the control parameters of the simulation
        
            * **params["EXE"]** ( string ): path to the DFTB+ executable [ default: dftb+ ]
            * **params["mo_active_space"]** ( list of ints or None ): indices of the MOs we care about 
                The indexing starts from 0, not 1! If set to None - all MOs will be returned. [default: None]
            * **params["md_file"]** ( string ): the name of the xyz file containing the trajectory - the 
                file should be in the xyz format produced by the DFTB+ program. [default: "md.xyz"]
            * **params["sp_gen_file"]** ( string ): the name of the .gen file that is listed in the 
                DFTB+ input file and

Let's do similar calculations, but for the next geometry, 1:

In [4]:
E1, MO1, H1, S1 = step2.do_step(1, params)

|===============================================================================
|
|  DFTB+ development version (commit: 3f79963e, base: 22.2)
|
|  Copyright (C) 2006 - 2022  DFTB+ developers group
|
|===============================================================================
|
|  When publishing results obtained with DFTB+, please cite the following
|  reference:
|
|  * DFTB+, a software package for efficient approximate density functional
|    theory based atomistic simulations, J. Chem. Phys. 152, 124101 (2020).
|    [doi: 10.1063/1.5143190]
|
|  You should also cite additional publications crediting the parametrization
|  data you use. Please consult the documentation of the SK-files for the
|  references.
|
|===============================================================================

Reading input file 'dftb_in.hsd'
Parser version: 13

--------------------------------------------------------------------------------
***  Converting input from parser version  4 to parser ver

ERROR STOP 

Error termination. Backtrace:
#0  0x559dd4a0aa8e in __dftbp_common_globalenv_MOD_abortprogram
	at /home/alexvakimov/SOFTWARE/dftbplus/dftbplus_22.2/src/dftbp/common/globalenv.F90:195
#1  0x559dd4adaa9a in __dftbp_dftbplus_mainio_MOD_writehsandstop
	at /home/alexvakimov/SOFTWARE/dftbplus/dftbplus_22.2/src/dftbp/dftbplus/mainio.F90:4166
#2  0x559dd4ac1547 in __dftbp_dftbplus_main_MOD_processgeometry
	at /home/alexvakimov/SOFTWARE/dftbplus/dftbplus_22.2/src/dftbp/dftbplus/main.F90:1262
#3  0x559dd4ac6351 in __dftbp_dftbplus_main_MOD_rundftbplus
	at /home/alexvakimov/SOFTWARE/dftbplus/dftbplus_22.2/src/dftbp/dftbplus/main.F90:236
#4  0x559dd4a09b85 in dftbplus
	at /home/alexvakimov/SOFTWARE/dftbplus/dftbplus_22.2/app/dftb+/dftbplus.F90:35
#5  0x559dd4a04fae in main
	at /home/alexvakimov/SOFTWARE/dftbplus/dftbplus_22.2/app/dftb+/dftbplus.F90:11


Orbitals, energies, etc. can be printed out to the files:

In [5]:
E0.show_matrix("E0.txt")
E1.show_matrix("E1.txt")
MO0.show_matrix("MO0.txt")
MO1.show_matrix("MO1.txt")

Now, let's compute the AO time-overlaps for these two geometries.

Note that we are using the same set of parameters as for the `do_step`. One of the keys is particularly important here (and wasn't used by the `do_step` function) - `"ovlp_in_file"`.

The first parameter of the function (0) is the index of the first geometry. The second one is adjacent (so 1).
<a name="do_ovlp-1"></a>

In [6]:
S01, S00, S11 = step2.do_ovlp(0, params)

|===============================================================================
|
|  DFTB+ development version (commit: 3f79963e, base: 22.2)
|
|  Copyright (C) 2006 - 2022  DFTB+ developers group
|
|===============================================================================
|
|  When publishing results obtained with DFTB+, please cite the following
|  reference:
|
|  * DFTB+, a software package for efficient approximate density functional
|    theory based atomistic simulations, J. Chem. Phys. 152, 124101 (2020).
|    [doi: 10.1063/1.5143190]
|
|  You should also cite additional publications crediting the parametrization
|  data you use. Please consult the documentation of the SK-files for the
|  references.
|
|===============================================================================

Reading input file 'dftb_in.hsd'
Parser version: 13

--------------------------------------------------------------------------------
***  Converting input from parser version  4 to parser ver

Note: The following floating-point exceptions are signalling: IEEE_UNDERFLOW_FLAG IEEE_DENORMAL
ERROR STOP 

Error termination. Backtrace:
#0  0x55a4da810a8e in __dftbp_common_globalenv_MOD_abortprogram
	at /home/alexvakimov/SOFTWARE/dftbplus/dftbplus_22.2/src/dftbp/common/globalenv.F90:195
#1  0x55a4da8e0a9a in __dftbp_dftbplus_mainio_MOD_writehsandstop
	at /home/alexvakimov/SOFTWARE/dftbplus/dftbplus_22.2/src/dftbp/dftbplus/mainio.F90:4166
#2  0x55a4da8c7547 in __dftbp_dftbplus_main_MOD_processgeometry
	at /home/alexvakimov/SOFTWARE/dftbplus/dftbplus_22.2/src/dftbp/dftbplus/main.F90:1262
#3  0x55a4da8cc351 in __dftbp_dftbplus_main_MOD_rundftbplus
	at /home/alexvakimov/SOFTWARE/dftbplus/dftbplus_22.2/src/dftbp/dftbplus/main.F90:236
#4  0x55a4da80fb85 in dftbplus
	at /home/alexvakimov/SOFTWARE/dftbplus/dftbplus_22.2/app/dftb+/dftbplus.F90:35
#5  0x55a4da80afae in main
	at /home/alexvakimov/SOFTWARE/dftbplus/dftbplus_22.2/app/dftb+/dftbplus.F90:11


A more detailed explanation of the function can be found here:

In [7]:
help(step2.do_ovlp)

Help on function do_ovlp in module libra_py.workflows.nbra.step2_dftb:

do_ovlp(snap, params)
    Compute the overlap matrix in the AO basis for two geometries, i and i+1
    
    Args:
        snap ( int ): index of the time step to be used from the trajectory file
        params ( dictionary ): the control parameters of the simulation
        
            * **params["EXE"]** ( string ): path to the DFTB+ executable [ default: dftb+ ]
            * **params["md_file"]** ( string ): the name of the xyz file containing the trajectory - the 
                file should be in the xyz format produced by the DFTB+ program. [default: "md.xyz"]
            * **params["ovlp_gen_file"]** ( string ): the name of the .gen file that is listed in the 
                DFTB+ input file and contains the geometry of the system at two time steps
                (the content of this file will be updated for every i value). [default: "x2.gen" ]
            * **params["syst_spec"]** ( string ): the string 

The function returns the off-diagonal block first, which corresponds to the AO time-overlap between two geometries. The other two returned objects are the diagonal blocks, so the normal AO overlap matrices for the two geometries separately.

With the AO overlaps and the cooresponding MO coefficients, we can check the orthogonality of the MOs:

In [8]:
I00 = MO0.H() * S00 * MO0
I11 = MO1.H() * S11 * MO1

I00.show_matrix("I00.txt")
I11.show_matrix("I11.txt")

If the corresponding .txt files contain diagonal matrices, with 1.0 on the diagonal and zeroes everywhere else, we got all the calculations right. 

Then, it is time to consider the time-overlap of the MOs:

$$ <\psi_i(t) | \psi_j(t+\Delta t) > = \sum_a {  \sum_b C_{ai}^* <\chi_a(t) | \chi_b(t+\Delta t) > C_{bj}}  $$

or in matrix notation

$$ S^{MO}(t; t+\Delta t) = C^\dagger (t) S^{AO}(t; t+\Delta t)  C(t+\Delta t) $$

where 

$$ S^{MO}_{ij}(t; t+\Delta t) = <\psi_i(t) | \psi_j(t+\Delta t) > $$

$$ S^{AO}_{ij}(t; t+\Delta t) = <\chi_i(t) | \chi_j(t+\Delta t) > $$

In [9]:
S01.show_matrix("AO_time_overlap.txt")

In [10]:
time_overlap01 = MO0.H() * S01 * MO1
time_overlap01.show_matrix("MO_time_overlap.txt")

NACs are then computed according the HST formula:

$$  NAC_{ij}(t+\frac{\Delta t}{2}) = \frac{ S^{MO}(t; t+\Delta t) - (S^{MO}(t; t+\Delta t))^\dagger } {2 \Delta t} $$

Here we assume `dt = 1.0`

In [11]:
NAC = (time_overlap01 - time_overlap01.H()) / (2.0 * 1.0)
NAC.show_matrix("NAC.txt")

## 3. NBRA workflow<a name="3"></a>
[Back to TOC](#toc)

The section above demonstrated the idea of computing time-overlaps and NACs. The extraction of state (MO) energies is a trivial task. So, all these operations can be combined in a workflow to compute multiple steps:
<a name="run_step2-1"></a>

In [12]:
prms = dict(params)

# Assume the trajectory was obtained with dt = 1 fs = 41 a.u.

prms.update({ "dt":41.0, "isnap":0, "fsnap":10, "out_dir":"res" })
step2.run_step2(prms)

|===============================================================================
|
|  DFTB+ development version (commit: 3f79963e, base: 22.2)
|
|  Copyright (C) 2006 - 2022  DFTB+ developers group
|
|===============================================================================
|
|  When publishing results obtained with DFTB+, please cite the following
|  reference:
|
|  * DFTB+, a software package for efficient approximate density functional
|    theory based atomistic simulations, J. Chem. Phys. 152, 124101 (2020).
|    [doi: 10.1063/1.5143190]
|
|  You should also cite additional publications crediting the parametrization
|  data you use. Please consult the documentation of the SK-files for the
|  references.
|
|===============================================================================

Reading input file 'dftb_in.hsd'
Parser version: 13

--------------------------------------------------------------------------------
***  Converting input from parser version  4 to parser ver

|===============================================================================
|
|  DFTB+ development version (commit: 3f79963e, base: 22.2)
|
|  Copyright (C) 2006 - 2022  DFTB+ developers group
|
|===============================================================================
|
|  When publishing results obtained with DFTB+, please cite the following
|  reference:
|
|  * DFTB+, a software package for efficient approximate density functional
|    theory based atomistic simulations, J. Chem. Phys. 152, 124101 (2020).
|    [doi: 10.1063/1.5143190]
|
|  You should also cite additional publications crediting the parametrization
|  data you use. Please consult the documentation of the SK-files for the
|  references.
|
|===============================================================================

Reading input file 'dftb_in.hsd'
Parser version: 13

--------------------------------------------------------------------------------
***  Converting input from parser version  4 to parser ver

ERROR STOP 

Error termination. Backtrace:
#0  0x5561aad76a8e in __dftbp_common_globalenv_MOD_abortprogram
	at /home/alexvakimov/SOFTWARE/dftbplus/dftbplus_22.2/src/dftbp/common/globalenv.F90:195
#1  0x5561aae46a9a in __dftbp_dftbplus_mainio_MOD_writehsandstop
	at /home/alexvakimov/SOFTWARE/dftbplus/dftbplus_22.2/src/dftbp/dftbplus/mainio.F90:4166
#2  0x5561aae2d547 in __dftbp_dftbplus_main_MOD_processgeometry
	at /home/alexvakimov/SOFTWARE/dftbplus/dftbplus_22.2/src/dftbp/dftbplus/main.F90:1262
#3  0x5561aae32351 in __dftbp_dftbplus_main_MOD_rundftbplus
	at /home/alexvakimov/SOFTWARE/dftbplus/dftbplus_22.2/src/dftbp/dftbplus/main.F90:236
#4  0x5561aad75b85 in dftbplus
	at /home/alexvakimov/SOFTWARE/dftbplus/dftbplus_22.2/app/dftb+/dftbplus.F90:35
#5  0x5561aad70fae in main
	at /home/alexvakimov/SOFTWARE/dftbplus/dftbplus_22.2/app/dftb+/dftbplus.F90:11


Done.


Processed input in HSD format written to 'dftb_pin.hsd'

Starting initialization...
--------------------------------------------------------------------------------
OpenMP threads:              16
Chosen random seed:          595597492
Current stacksize:           8 [Mb] (recommended: unlimited)
WARNING!
-> Current stacksize not set to unlimited or hard limit, which might cause
   random crashes (e.g. segmentation faults). It is advised to unlimit the
   stacksize by issuing 'ulimit -s unlimited' (Linux) or setting it to the 
   hard limit by 'ulimit -s hard' (Mac) in advance.
Mode:                        Static calculation
Self consistent charges:     Yes
SCC-tolerance:                 0.100000E-04
Max. scc iterations:                    100
Shell resolved Hubbard:      No
Spin polarisation:           No
Nr. of up electrons:            28.000000
Nr. of down electrons:          28.000000
Periodic boundaries:         No
Electronic solver:           Relatively robust
Mixer:      

ERROR STOP 

Error termination. Backtrace:
#0  0x556ea6e4fa8e in __dftbp_common_globalenv_MOD_abortprogram
	at /home/alexvakimov/SOFTWARE/dftbplus/dftbplus_22.2/src/dftbp/common/globalenv.F90:195
#1  0x556ea6f1fa9a in __dftbp_dftbplus_mainio_MOD_writehsandstop
	at /home/alexvakimov/SOFTWARE/dftbplus/dftbplus_22.2/src/dftbp/dftbplus/mainio.F90:4166
#2  0x556ea6f06547 in __dftbp_dftbplus_main_MOD_processgeometry
	at /home/alexvakimov/SOFTWARE/dftbplus/dftbplus_22.2/src/dftbp/dftbplus/main.F90:1262
#3  0x556ea6f0b351 in __dftbp_dftbplus_main_MOD_rundftbplus
	at /home/alexvakimov/SOFTWARE/dftbplus/dftbplus_22.2/src/dftbp/dftbplus/main.F90:236
#4  0x556ea6e4eb85 in dftbplus
	at /home/alexvakimov/SOFTWARE/dftbplus/dftbplus_22.2/app/dftb+/dftbplus.F90:35
#5  0x556ea6e49fae in main
	at /home/alexvakimov/SOFTWARE/dftbplus/dftbplus_22.2/app/dftb+/dftbplus.F90:11


...

Note: The following floating-point exceptions are signalling: IEEE_UNDERFLOW_FLAG IEEE_DENORMAL
ERROR STOP 

Error termination. Backtrace:
#0  0x55e8a0368a8e in __dftbp_common_globalenv_MOD_abortprogram
	at /home/alexvakimov/SOFTWARE/dftbplus/dftbplus_22.2/src/dftbp/common/globalenv.F90:195
#1  0x55e8a0438a9a in __dftbp_dftbplus_mainio_MOD_writehsandstop
	at /home/alexvakimov/SOFTWARE/dftbplus/dftbplus_22.2/src/dftbp/dftbplus/mainio.F90:4166
#2  0x55e8a041f547 in __dftbp_dftbplus_main_MOD_processgeometry
	at /home/alexvakimov/SOFTWARE/dftbplus/dftbplus_22.2/src/dftbp/dftbplus/main.F90:1262
#3  0x55e8a0424351 in __dftbp_dftbplus_main_MOD_rundftbplus
	at /home/alexvakimov/SOFTWARE/dftbplus/dftbplus_22.2/src/dftbp/dftbplus/main.F90:236
#4  0x55e8a0367b85 in dftbplus
	at /home/alexvakimov/SOFTWARE/dftbplus/dftbplus_22.2/app/dftb+/dftbplus.F90:35
#5  0x55e8a0362fae in main
	at /home/alexvakimov/SOFTWARE/dftbplus/dftbplus_22.2/app/dftb+/dftbplus.F90:11


|===============================================================================
|
|  DFTB+ development version (commit: 3f79963e, base: 22.2)
|
|  Copyright (C) 2006 - 2022  DFTB+ developers group
|
|===============================================================================
|
|  When publishing results obtained with DFTB+, please cite the following
|  reference:
|
|  * DFTB+, a software package for efficient approximate density functional
|    theory based atomistic simulations, J. Chem. Phys. 152, 124101 (2020).
|    [doi: 10.1063/1.5143190]
|
|  You should also cite additional publications crediting the parametrization
|  data you use. Please consult the documentation of the SK-files for the
|  references.
|
|===============================================================================

Reading input file 'dftb_in.hsd'
Parser version: 13

--------------------------------------------------------------------------------
***  Converting input from parser version  4 to parser ver

ERROR STOP 

Error termination. Backtrace:
#0  0x55fccaefca8e in __dftbp_common_globalenv_MOD_abortprogram
	at /home/alexvakimov/SOFTWARE/dftbplus/dftbplus_22.2/src/dftbp/common/globalenv.F90:195
#1  0x55fccafcca9a in __dftbp_dftbplus_mainio_MOD_writehsandstop
	at /home/alexvakimov/SOFTWARE/dftbplus/dftbplus_22.2/src/dftbp/dftbplus/mainio.F90:4166
#2  0x55fccafb3547 in __dftbp_dftbplus_main_MOD_processgeometry
	at /home/alexvakimov/SOFTWARE/dftbplus/dftbplus_22.2/src/dftbp/dftbplus/main.F90:1262
#3  0x55fccafb8351 in __dftbp_dftbplus_main_MOD_rundftbplus
	at /home/alexvakimov/SOFTWARE/dftbplus/dftbplus_22.2/src/dftbp/dftbplus/main.F90:236
#4  0x55fccaefbb85 in dftbplus
	at /home/alexvakimov/SOFTWARE/dftbplus/dftbplus_22.2/app/dftb+/dftbplus.F90:35
#5  0x55fccaef6fae in main
	at /home/alexvakimov/SOFTWARE/dftbplus/dftbplus_22.2/app/dftb+/dftbplus.F90:11




Processed input in HSD format written to 'dftb_pin.hsd'

Starting initialization...
--------------------------------------------------------------------------------
OpenMP threads:              16
Chosen random seed:          595610208
Current stacksize:           8 [Mb] (recommended: unlimited)
WARNING!
-> Current stacksize not set to unlimited or hard limit, which might cause
   random crashes (e.g. segmentation faults). It is advised to unlimit the
   stacksize by issuing 'ulimit -s unlimited' (Linux) or setting it to the 
   hard limit by 'ulimit -s hard' (Mac) in advance.
Mode:                        Static calculation
Self consistent charges:     Yes
SCC-tolerance:                 0.100000E-04
Max. scc iterations:                    100
Shell resolved Hubbard:      No
Spin polarisation:           No
Nr. of up electrons:            56.000000
Nr. of down electrons:          56.000000
Periodic boundaries:         No
Electronic solver:           Relatively robust
Mixer:            

Note: The following floating-point exceptions are signalling: IEEE_UNDERFLOW_FLAG IEEE_DENORMAL
ERROR STOP 

Error termination. Backtrace:
#0  0x562a35fcaa8e in __dftbp_common_globalenv_MOD_abortprogram
	at /home/alexvakimov/SOFTWARE/dftbplus/dftbplus_22.2/src/dftbp/common/globalenv.F90:195
#1  0x562a3609aa9a in __dftbp_dftbplus_mainio_MOD_writehsandstop
	at /home/alexvakimov/SOFTWARE/dftbplus/dftbplus_22.2/src/dftbp/dftbplus/mainio.F90:4166
#2  0x562a36081547 in __dftbp_dftbplus_main_MOD_processgeometry
	at /home/alexvakimov/SOFTWARE/dftbplus/dftbplus_22.2/src/dftbp/dftbplus/main.F90:1262
#3  0x562a36086351 in __dftbp_dftbplus_main_MOD_rundftbplus
	at /home/alexvakimov/SOFTWARE/dftbplus/dftbplus_22.2/src/dftbp/dftbplus/main.F90:236
#4  0x562a35fc9b85 in dftbplus
	at /home/alexvakimov/SOFTWARE/dftbplus/dftbplus_22.2/app/dftb+/dftbplus.F90:35
#5  0x562a35fc4fae in main
	at /home/alexvakimov/SOFTWARE/dftbplus/dftbplus_22.2/app/dftb+/dftbplus.F90:11


Cleanup after the endire project:

In [14]:
def do_cleanup():
    tmp = ["*.gen", "*.txt", "res", "*.bin", "*.out", "*.dat", "*.dat_tmp", "*.tag", "dftb_in.hsd", "dftb_pin.hsd"]
    for item in tmp:
        os.system(F"rm -r {item}")

#do_cleanup()